In [14]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score,mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import os
from sklearn.impute import SimpleImputer
from sklearn. linear_model import LogisticRegression

In [15]:
df=pd.read_csv("train.csv")

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159256 entries, 0 to 159255
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   159256 non-null  int64  
 1   age                  159256 non-null  int64  
 2   height(cm)           159256 non-null  int64  
 3   weight(kg)           159256 non-null  int64  
 4   waist(cm)            159256 non-null  float64
 5   eyesight(left)       159256 non-null  float64
 6   eyesight(right)      159256 non-null  float64
 7   hearing(left)        159256 non-null  int64  
 8   hearing(right)       159256 non-null  int64  
 9   systolic             159256 non-null  int64  
 10  relaxation           159256 non-null  int64  
 11  fasting blood sugar  159256 non-null  int64  
 12  Cholesterol          159256 non-null  int64  
 13  triglyceride         159256 non-null  int64  
 14  HDL                  159256 non-null  int64  
 15  LDL              

In [17]:
df.isna().sum().sum()

0

In [18]:
df.set_index("id", inplace = True)

In [19]:
x=df.drop("smoking",axis=1)
y=df["smoking"]

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

params= {
    'penalty': ['l2',None], "C":np.linspace(0.001,10,20), # Try both L1 and L2 penalties
    'solver': ['lbfgs', 'newton-cg','newton-cholesky',"sag"]}
lr=LogisticRegression()
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=24)
gcv=GridSearchCV(lr,param_grid=params,cv=kfold,scoring="neg_log_loss")
gcv.fit(x,y)
print(gcv.best_params_)
print(gcv.best_score_)

C:\Users\dbda\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\dbda\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stab

KeyboardInterrupt: 

In [ ]:
## inferencing
test=pd.read_csv("test.csv",index_col=0)
best_model=gcv.best_estimator_
y_pred_prob=best_model.predict_proba(test)
submit=pd.read_csv("sample_submission.csv")
submit["smoking"]=y_pred_prob[:,1]
submit.to_csv("submission_f.csv",index=False)

## using scaling all scalling

In [21]:
from sklearn.metrics import log_loss

from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

mm_scaler=MinMaxScaler()
std_scaler=StandardScaler()
knn=KNeighborsClassifier()


params={"KNN__n_neighbors":np.arange(1,40),"SCL":[None,std_scaler,mm_scaler]}

pipe=Pipeline([("SCL",None),("KNN",knn)])

kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=24)

gcv=GridSearchCV(pipe,param_grid=params,cv=kfold,verbose=3,scoring="neg_log_loss")  #verbose by default is zero, 0=going to know updates of after execution, 1= 
gcv.fit(x,y)

print(gcv.best_params_)
print(gcv.best_score_)

Fitting 5 folds for each of 117 candidates, totalling 585 fits
[CV 1/5] END ....KNN__n_neighbors=1, SCL=None;, score=-11.594 total time=   6.5s
[CV 2/5] END ....KNN__n_neighbors=1, SCL=None;, score=-11.571 total time=   6.5s
[CV 3/5] END ....KNN__n_neighbors=1, SCL=None;, score=-11.496 total time=   6.1s
[CV 4/5] END ....KNN__n_neighbors=1, SCL=None;, score=-11.463 total time=   5.8s
[CV 5/5] END ....KNN__n_neighbors=1, SCL=None;, score=-11.589 total time=   6.0s
[CV 1/5] END KNN__n_neighbors=1, SCL=StandardScaler();, score=-10.984 total time=   6.0s
[CV 2/5] END KNN__n_neighbors=1, SCL=StandardScaler();, score=-10.994 total time=   5.7s
[CV 3/5] END KNN__n_neighbors=1, SCL=StandardScaler();, score=-11.066 total time=   6.8s
[CV 4/5] END KNN__n_neighbors=1, SCL=StandardScaler();, score=-11.175 total time=   7.1s
[CV 5/5] END KNN__n_neighbors=1, SCL=StandardScaler();, score=-11.196 total time=   6.7s
[CV 1/5] END KNN__n_neighbors=1, SCL=MinMaxScaler();, score=-11.324 total time=   6.5s


In [22]:
## inferencing
test=pd.read_csv("test.csv",index_col=0)
best_model=gcv.best_estimator_
y_pred_prob=best_model.predict_proba(test)
submit=pd.read_csv("sample_submission.csv")
submit["smoking"]=y_pred_prob[:,1]
submit.to_csv("submission_f11.csv",index=False)